# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# general + data loading
import pandas as pd
from sqlalchemy import create_engine

# pre-processing
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# machine learning
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# storage
import pickle

In [11]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_data.db')
df = pd.read_sql_table('texts', engine)
df.head()
X = df[['message', 'original', 'genre']]
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [13]:
stops = set(stopwords.words('english'))

def tokenize(text):
    # remove punctuation
    text = re.sub("[\.\\:;!?'\"-]", " ", text.lower())
    tokens = word_tokenize(text)
    
    # remove stopwords
    tokens = [tok for tok in tokens if tok not in stops]
    
    # part of speech
    tags = pos_tag(tokens)
    
    # TODO: further processing (e.g. NER)
    return tags
    
X['message'].head().apply(tokenize)

0    [(weather, NN), (update, JJ), (cold, JJ), (fro...
1                                    [(hurricane, NN)]
2          [(looking, VBG), (someone, NN), (name, NN)]
3    [(un, JJ), (reports, NNS), (leogane, VBD), (80...
4    [(says, VBZ), (west, JJS), (side, NN), (haiti,...
Name: message, dtype: object

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline = [
    ('vectorize', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('cls', MultiOutputClassifier(SVC(), n_jobs=1) )
]
pipeline = Pipeline(pipeline)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
pipeline.fit(X_train['message'], y_train)

c:\programming\envs\datascience\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\programming\envs\datascience\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\programming\envs\datascience\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\programming\envs\datascience\lib\site-packages\sklearn\

Pipeline(memory=None,
         steps=[('vectorize',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function token...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('cls',
                 MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200,
                                                     class_weight=None,
                                

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = pipeline.predict(X_test['message'])
f1_score(y_test, y_pred, average='micro')

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted', 'samples'].

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
f1_score(y_test, y_pred, average='micro')

0.3671530161844041

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
filename = '../models/pipeline.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.